In [28]:
import pandas as pd
from pybedtools import BedTool

def compute_ovlp_len( row):
    if row['SE_name'] == '.':
        return 0
    value = min(row['end'], row['SE_end']) - max(row['start'], row['SE_start'])
    return value

def compute_ovlp_pct( row):
    size = abs(row['end'] - row['start'])
    value = row['SE_ovlp_len'] / size * 100.0
    return value


assembly='hg19'
method='DNase_H3K27ac'
min_overlap=0.1

staging_path = "/Users/manuel/development/thesis/staging"
overlap_path = "/Users/manuel/development/thesis/overlap"
encode_file_name = "filtered_"

if assembly:
    encode_file_name += assembly
if method:
    encode_file_name += method

encode_file_path = staging_path + "/ENCODE/filtered/" + encode_file_name + ".csv"
dbsuper_file_path = staging_path + "/dbSUPER/adrenal_gland_dbSUPER_sample_full.csv"
encode_bed_file_path = staging_path + "/dbSUPER/adrenal_gland_ENCODE_sample.bed"
dbsuper_bed_filepath = staging_path + "/dbSUPER/adrenal_gland_dbSUPER_sample.bed"

encode_bed = BedTool(encode_bed_file_path)
dbsuper_bed = BedTool(dbsuper_bed_filepath)

print("ENCODE bed file:", encode_bed_file_path)
print("dbSUPER bed file:", dbsuper_bed_filepath)
print("Starting overlap...")

# full_dbSUPER_intersect_ENCODE = dbsuper_bed.intersect(encode_bed, wa=True, f=0.1)
encode_intersect_dbsuper = encode_bed.intersect(dbsuper_bed, loj=True, f=min_overlap)

# print(len(full_dbSUPER_intersect_ENCODE), "dbSUPER.intersect(ENCODE) results")
print(len(encode_intersect_dbsuper), "ENCODE.intersect(dbSUPER) results")

overlap_column_names = ['chrom', 'start', 'end', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb',
                        'SE_chrom', \
                        'SE_start', 'SE_end', 'SE_name', 'SE_score']
overlap_df = encode_intersect_dbsuper.to_dataframe(names=overlap_column_names)

print("dbSUPER details file:", dbsuper_file_path)
full_dbsuper_details_df = pd.read_csv(dbsuper_file_path, sep=',')

print("Merging details from dbSUPER...")
overlap_full_detail_df = overlap_df.merge(
    full_dbsuper_details_df[['ID', 'Size', 'Associated Gene', 'Method', 'Rank', 'Cell/Tissue']],
    how='left', left_on='SE_name', right_on='ID')

overlap_full_detail_df = overlap_full_detail_df.drop('thickStart', axis=1)
overlap_full_detail_df = overlap_full_detail_df.drop('thickEnd', axis=1)
overlap_full_detail_df = overlap_full_detail_df.drop('itemRgb', axis=1)
overlap_full_detail_df = overlap_full_detail_df.drop('ID', axis=1)
overlap_full_detail_df = overlap_full_detail_df.drop('Rank', axis=1)

overlap_full_detail_df.columns = ['chrom', 'start', 'end', 'name', 'score', 'strand', 'SE_chrom',
                                  'SE_start', 'SE_end', 'SE_name', 'SE_score', 'SE_size',
                                  'SE_associated_gene', 'SE_method', 'SE_biosample']

overlap_full_detail_df['SE_ovlp_len'] = overlap_full_detail_df.apply(lambda row: compute_ovlp_len(row),
                                                                     axis=1)
overlap_full_detail_df['SE_ovlp_pct'] = overlap_full_detail_df.apply(lambda row: compute_ovlp_pct(row),
                                                                     axis=1)

overlap_full_detail_df['SE_associated_gene'].fillna('.', inplace=True)
overlap_full_detail_df['SE_method'].fillna('.', inplace=True)
overlap_full_detail_df['SE_biosample'].fillna('.', inplace=True)
#overlap_full_detail_df['SE_size'].fillna('-1', inplace=True)



ENCODE bed file: /Users/manuel/development/thesis/staging/dbSUPER/adrenal_gland_ENCODE_sample.bed
dbSUPER bed file: /Users/manuel/development/thesis/staging/dbSUPER/adrenal_gland_dbSUPER_sample.bed
Starting overlap...
38634 ENCODE.intersect(dbSUPER) results
dbSUPER details file: /Users/manuel/development/thesis/staging/dbSUPER/adrenal_gland_dbSUPER_sample_full.csv
Merging details from dbSUPER...


In [29]:
overlap_full_detail_df['name'] = 'ENCODE.3.ENCFF778PVS.0'
overlap_full_detail_df.T

,0,1,2,3,4,5,6,7,8,9,...,38624,38625,38626,38627,38628,38629,38630,38631,38632,38633
chrom,chr13,chr14,chr16,chr14,chr12,chr19,chr17,chr19,chr7,chr19,...,chr19,chr19,chr17,chr19,chr12,chr5,chr16,chr11,chr18,chr16
start,34114318,35800256,22200146,62025090,8178209,18413303,41436643,2057975,130008428,39891316,...,46880463,45260914,52847987,11465980,54820681,121647840,67193280,44117140,24127860,11762160
end,34118911,35812312,22205387,62033524,8180468,18418380,41439942,2063141,130015073,39895380,...,46882005,45261457,52848805,11466130,54823625,121647990,67194085,44117290,24128010,11762310
name,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,...,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0
score,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
strand,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
SE_chrom,.,chr14,.,chr14,.,.,.,chr19,.,chr19,...,.,chr19,.,.,.,.,.,.,.,.
SE_start,-1,35800648,-1,61977862,-1,-1,-1,2031672,-1,39887759,...,-1,45240079,-1,-1,-1,-1,-1,-1,-1,-1
SE_end,-1,35813368,-1,62062455,-1,-1,-1,2094175,-1,39902523,...,-1,45263101,-1,-1,-1,-1,-1,-1,-1,-1
SE_name,.,SE_01165,.,SE_00951,.,.,.,SE_00877,.,SE_01016,...,.,SE_01042,.,.,.,.,.,.,.,.


In [30]:
print("ENCODE details file:", encode_file_path)
encode_details_df = pd.read_csv(encode_file_path, sep='\t')


ENCODE details file: /Users/manuel/development/thesis/staging/ENCODE/filtered/filtered_hg19DNase_H3K27ac.csv


In [31]:
encode_details_df.T

,0,1,2,3,4,5,6,7,8,9,...,1785570,1785571,1785572,1785573,1785574,1785575,1785576,1785577,1785578,1785579
chrom,chr3,chr1,chr15,chr5,chr11,chr5,chr10,chr10,chr3,chr8,...,chr16,chr1,chr6,chr19,chr7,chr18,chr1,chr5,chr6,chr17
start,152855118,214611302,101260386,52654619,9586536,148864272,3892558,5624645,5062817,126230865,...,58528640,60138160,105771787,8428130,24613000,20807015,150485494,178450920,155442359,37309960
end,152861069,214622352,101269291,52660930,9592486,148869801,3895911,5628826,5068862,126234434,...,58529655,60139325,105773337,8429430,24613150,20808529,150486475,178451070,155443585,37310110
name,Distal-Prediction-1,Distal-Prediction-2,Distal-Prediction-3,Distal-Prediction-4,Distal-Prediction-5,Distal-Prediction-6,Distal-Prediction-7,Distal-Prediction-8,Distal-Prediction-9,Distal-Prediction-10,...,Proximal-Prediction-14000,Proximal-Prediction-14001,Proximal-Prediction-14002,Proximal-Prediction-14003,Proximal-Prediction-14004,Proximal-Prediction-14005,Proximal-Prediction-14006,Proximal-Prediction-14007,Proximal-Prediction-14008,Proximal-Prediction-14009
score,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
strand,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
candidate_id,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.1,ENCODE.3.ENCFF778PVS.2,ENCODE.3.ENCFF778PVS.3,ENCODE.3.ENCFF778PVS.4,ENCODE.3.ENCFF778PVS.5,ENCODE.3.ENCFF778PVS.6,ENCODE.3.ENCFF778PVS.7,ENCODE.3.ENCFF778PVS.8,ENCODE.3.ENCFF778PVS.9,...,ENCODE.3.ENCFF026HMJ.33999,ENCODE.3.ENCFF026HMJ.34000,ENCODE.3.ENCFF026HMJ.34001,ENCODE.3.ENCFF026HMJ.34002,ENCODE.3.ENCFF026HMJ.34003,ENCODE.3.ENCFF026HMJ.34004,ENCODE.3.ENCFF026HMJ.34005,ENCODE.3.ENCFF026HMJ.34006,ENCODE.3.ENCFF026HMJ.34007,ENCODE.3.ENCFF026HMJ.34008
assembly,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19,...,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19,hg19
biosample_term_id,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,CL:0002327,...,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372,CL:0002372
biosample_term_name,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,mammary epithelial cell,...,myotube,myotube,myotube,myotube,myotube,myotube,myotube,myotube,myotube,myotube


In [32]:

print("Merging details from ENCODE...")
overlap_full_detail_encode_df = overlap_full_detail_df.merge(
    encode_details_df[['candidate_id', 'assembly', 'biosample_term_id', 'biosample_term_name', 'biosample_type',
                       'description', 'developmental_slims', 'encyclopedia', 'encyclopedia_version',
                       'organ_slims', 'system_slims', 'method']],
    how='left', left_on='name', right_on='candidate_id'
)

Merging details from ENCODE...


In [33]:
overlap_full_detail_encode_df.T

,0,1,2,3,4,5,6,7,8,9,...,38624,38625,38626,38627,38628,38629,38630,38631,38632,38633
chrom,chr13,chr14,chr16,chr14,chr12,chr19,chr17,chr19,chr7,chr19,...,chr19,chr19,chr17,chr19,chr12,chr5,chr16,chr11,chr18,chr16
start,34114318,35800256,22200146,62025090,8178209,18413303,41436643,2057975,130008428,39891316,...,46880463,45260914,52847987,11465980,54820681,121647840,67193280,44117140,24127860,11762160
end,34118911,35812312,22205387,62033524,8180468,18418380,41439942,2063141,130015073,39895380,...,46882005,45261457,52848805,11466130,54823625,121647990,67194085,44117290,24128010,11762310
name,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,...,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0
score,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
strand,.,.,.,.,.,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,.
SE_chrom,.,chr14,.,chr14,.,.,.,chr19,.,chr19,...,.,chr19,.,.,.,.,.,.,.,.
SE_start,-1,35800648,-1,61977862,-1,-1,-1,2031672,-1,39887759,...,-1,45240079,-1,-1,-1,-1,-1,-1,-1,-1
SE_end,-1,35813368,-1,62062455,-1,-1,-1,2094175,-1,39902523,...,-1,45263101,-1,-1,-1,-1,-1,-1,-1,-1
SE_name,.,SE_01165,.,SE_00951,.,.,.,SE_00877,.,SE_01016,...,.,SE_01042,.,.,.,.,.,.,.,.


In [34]:
output_filename = overlap_path + "/" + encode_file_name + "_dbSUPER_overlapped_adrenal_gland.csv"
print("Exporting overlapped file to:", output_filename)
overlap_full_detail_encode_df.to_csv(output_filename, index=None, sep='\t')

print(overlap_full_detail_encode_df.head())

Exporting overlapped file to: /Users/manuel/development/thesis/overlap/filtered_hg19DNase_H3K27ac_dbSUPER_overlapped_adrenal_gland.csv
   chrom     start       end                    name  score strand SE_chrom  \
0  chr13  34114318  34118911  ENCODE.3.ENCFF778PVS.0      1      .        .   
1  chr14  35800256  35812312  ENCODE.3.ENCFF778PVS.0      1      .    chr14   
2  chr16  22200146  22205387  ENCODE.3.ENCFF778PVS.0      1      .        .   
3  chr14  62025090  62033524  ENCODE.3.ENCFF778PVS.0      1      .    chr14   
4  chr12   8178209   8180468  ENCODE.3.ENCFF778PVS.0      1      .        .   

   SE_start    SE_end   SE_name      ...        biosample_term_id  \
0        -1        -1         .      ...               CL:0002327   
1  35800648  35813368  SE_01165      ...               CL:0002327   
2        -1        -1         .      ...               CL:0002327   
3  61977862  62062455  SE_00951      ...               CL:0002327   
4        -1        -1         .      ...     

In [35]:
big_overlap_file = '/Users/manuel/development/thesis/overlap/filtered_hg19DNase_H3K27ac_dbSUPER_overlapped.csv'
import pandas as pd

with open(big_overlap_file) as myfile:
    head = [next(myfile).split('\t') for x in range(4)]

In [43]:
df = pd.DataFrame(head[1:4])
df.columns = head[0]
df.T

,0,1,2
chrom,chr3,chr3,chr1
start,152855118,152855118,214611302
end,152861069,152861069,214622352
name,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.0,ENCODE.3.ENCFF778PVS.1
score,1,1,1
strand,.,.,.
SE_chrom,chr3,chr3,chr1
SE_start,152847915,152847755,214594593
SE_end,152884602,152864205,214629357
SE_name,SE_35899,SE_64659,SE_02460
